# HW 4 - POS Tagging with Hugging Face

In this exercise, you will create a part-of-speech (POS) tagging system for Thai text using NECTEC’s ORCHID corpus. Instead of building your own deep learning architecture from scratch, you will leverage a pretrained tokenizer and a pretrained token classification model from Hugging Face.

We have provided some starter code for data cleaning and preprocessing in this notebook, but feel free to modify those parts to suit your needs. You are welcome to use additional libraries (e.g., scikit-learn) as long as you incorporate the pretrained Hugging Face model. Specifically, you will need to:

1. Load a pretrained tokenizer and token classification model.
2. Fine-tune it on the ORCHID corpus for POS tagging.
3. Evaluate and report the performance of your model on the test data.

### Don't forget to change hardware accelrator to GPU in runtime on Google Colab ###

## 1. Setup and Preprocessing

In [4]:
# # Install transformers and thai2transformers
# !pip install wandb
# !pip install -q transformers==4.30.1 datasets evaluate thaixtransformers
# !pip install -q emoji pythainlp sefr_cut tinydb seqeval sentencepiece pydantic jsonlines
# !pip install peft==0.10.0

## Setup

1. Register [Wandb account](https://wandb.ai/login?signup=true) (and confirm your email)

2. `wandb login` and copy paste the API key when prompt

In [5]:
# import wandb
# from kaggle_secrets import UserSecretsClient

# user_secrets = UserSecretsClient()

# my_secret = user_secrets.get_secret("wandb_api_key") 

# wandb.login(key=my_secret)

We encourage you to login to your `Hugging Face` account so you can upload and share your model with the community. When prompted, enter your token to login

In [6]:
# from huggingface_hub import notebook_login

# notebook_login()

Download the dataset from Hugging Face

In [7]:
from datasets import load_dataset

orchid = load_dataset("Thichow/orchid_corpus")

README.md:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

orchid_corpus.py:   0%|          | 0.00/7.91k [00:00<?, ?B/s]

The repository for Thichow/orchid_corpus contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Thichow/orchid_corpus.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
orchid

DatasetDict({
    train: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence'],
        num_rows: 4625
    })
})

In [9]:
orchid['train'][0]

{'id': '0',
 'label_tokens': ['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1'],
 'pos_tags': [21, 39, 26, 26, 37, 4, 18],
 'sentence': 'การประชุมทางวิชาการ ครั้งที่ 1'}

In [10]:
orchid['train'][0]["sentence"]

'การประชุมทางวิชาการ ครั้งที่ 1'

In [11]:
''.join(orchid['train'][0]['label_tokens'])

'การประชุมทางวิชาการ ครั้งที่ 1'

In [12]:
label_list = orchid["train"].features[f"pos_tags"].feature.names
print('total type of pos_tags :', len(label_list))
print(label_list)

total type of pos_tags : 47
['ADVI', 'ADVN', 'ADVP', 'ADVS', 'CFQC', 'CLTV', 'CMTR', 'CMTR@PUNC', 'CNIT', 'CVBL', 'DCNM', 'DDAC', 'DDAN', 'DDAQ', 'DDBQ', 'DIAC', 'DIAQ', 'DIBQ', 'DONM', 'EAFF', 'EITT', 'FIXN', 'FIXV', 'JCMP', 'JCRG', 'JSBR', 'NCMN', 'NCNM', 'NEG', 'NLBL', 'NONM', 'NPRP', 'NTTL', 'PDMN', 'PNTR', 'PPRS', 'PREL', 'PUNC', 'RPRE', 'VACT', 'VATT', 'VSTA', 'XVAE', 'XVAM', 'XVBB', 'XVBM', 'XVMM']


In [13]:
import numpy as np
import numpy.random
import torch

from tqdm.auto import tqdm
from functools import partial

#transformers
from transformers import (
    CamembertTokenizer,
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)

#thaixtransformers
from thaixtransformers import Tokenizer
from thaixtransformers.preprocess import process_transformers

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Next, we load a pretrained tokenizer from Hugging Face. In this work, we utilize WangchanBERTa, a Thai-specific pretrained model, as the tokenizer.

# Choose Pretrained Model

In this notebook, you can choose from 5 versions of WangchanBERTa, XLMR and mBERT to perform downstream tasks on Thai datasets. The datasets are:

* `wangchanberta-base-att-spm-uncased` (recommended) - Largest WangchanBERTa trained on 78.5GB of Assorted Thai Texts with subword tokenizer SentencePiece
* `xlm-roberta-base` - Facebook's [XLMR](https://arxiv.org/abs/1911.02116) trained on 100 languages
* `bert-base-multilingual-cased` - Google's [mBERT](https://arxiv.org/abs/1911.03310) trained on 104 languages
* `wangchanberta-base-wiki-newmm` - WangchanBERTa trained on Thai Wikipedia Dump with PyThaiNLP's word-level tokenizer  `newmm`
* `wangchanberta-base-wiki-syllable` - WangchanBERTa trained on Thai Wikipedia Dump with PyThaiNLP's syllabel-level tokenizer `syllable`
* `wangchanberta-base-wiki-sefr` - WangchanBERTa trained on Thai Wikipedia Dump with word-level tokenizer  `SEFR`
* `wangchanberta-base-wiki-spm` - WangchanBERTa trained on Thai Wikipedia Dump with subword-level tokenizer SentencePiece

In the first part, we require you to select the wangchanberta-base-att-spm-uncased.

<b> Learn more about using wangchanberta at [wangchanberta_getting_started_ai_reseach](https://colab.research.google.com/github/PyThaiNLP/thaixtransformers/blob/main/notebooks/wangchanberta_getting_started_aireseach.ipynb?fbclid=IwY2xjawH61XZleHRuA2FlbQIxMAABHZUaAmHobzmCMHpX0EgdLdjDAEwSX0bjqpo5xPUSIx9b4O_dsIvvG8KVNA_aem_IyKkvzy-VPf9k2pYAFf6Nw#scrollTo=n5IaCot9b3cF) <b>



*   You need to set the transformers version to transformers==4.30.1.



`In the first part, we require you to select the wangchanberta-base-att-spm-uncased.`

In [14]:
model_names = [
    'airesearch/wangchanberta-base-att-spm-uncased',
    'airesearch/wangchanberta-base-wiki-newmm',
    'airesearch/wangchanberta-base-wiki-ssg',
    'airesearch/wangchanberta-base-wiki-sefr',
    'airesearch/wangchanberta-base-wiki-spm',
]

#@title Choose Pretrained Model
model_name = "airesearch/wangchanberta-base-att-spm-uncased"

#create tokenizer
tokenizer = Tokenizer(model_name).from_pretrained(
                f'{model_name}',
                revision='main',
                model_max_length=416,)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.


Let's try using a pretrained tokenizer.

In [15]:
text = 'ศิลปะไม่เป็นเจ้านายใคร และไม่เป็นขี้ข้าใคร'
print('text :', text)
tokens = []
for i in tokenizer([text], is_split_into_words=True)['input_ids']:
  tokens.append(tokenizer.decode(i))
print('tokens :', tokens)

text : ศิลปะไม่เป็นเจ้านายใคร และไม่เป็นขี้ข้าใคร
tokens : ['<s>', '', 'ศิลปะ', 'ไม่เป็น', 'เจ้านาย', 'ใคร', '<_>', 'และ', 'ไม่เป็น', 'ขี้ข้า', 'ใคร', '</s>']


model : * `wangchanberta-base-att-spm-uncased`

First, we print examples of label tokens from our dataset for inspection.

In [16]:
example = orchid["train"][0]
for i in example :
    print(i, ':', example[i])

id : 0
label_tokens : ['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1']
pos_tags : [21, 39, 26, 26, 37, 4, 18]
sentence : การประชุมทางวิชาการ ครั้งที่ 1


Then, we use the sentence 'การประชุมทางวิชาการ<space>ครั้งที่ 1' to be tokenized by the pretrained tokenizer model.

In [17]:
text = 'การประชุมทางวิชาการ ครั้งที่ 1'
print(tokenizer(text))

{'input_ids': [5, 10, 882, 8222, 8, 10, 1014, 8, 10, 59, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


These are already mapped into discrete values. We can uncover the original token text from the tokens by.

In [18]:
for i in tokenizer(text)['input_ids']:
  print(tokenizer.convert_ids_to_tokens(i))

<s>
▁
การประชุม
ทางวิชาการ
<_>
▁
ครั้งที่
<_>
▁
1
</s>


Now let's look at another example.

In [19]:
example = orchid["train"][1899]
print('sentence :', example["sentence"])
tokenized_input = tokenizer([example["sentence"]], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print('tokens :',tokens,len(tokens))
print('label tokens :', example["label_tokens"],len(example["label_tokens"]))
print('label pos :', example["pos_tags"],len(example["pos_tags"]))

sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (Bilingual transfer dictionary)
tokens : ['<s>', '▁โดย', 'พิจารณาจาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '▁(', '<unk>', 'i', 'ling', 'ual', '<_>', '▁', 'trans', 'fer', '<_>', '▁', 'di', 'ction', 'ary', ')', '</s>'] 23
label tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'Bilingual transfer dictionary', ')'] 10
label pos : [25, 39, 38, 26, 26, 5, 37, 37, 26, 37] 10


Notice how `B` becomes an ``<unk>`` token. This is because this is an uncased model, meaning it only handles small English characters.

In [20]:
print("AAbdfsdfsSDFDSFด่ิอาห่สหด่สกห่ดสากหด่สหา".lower())

aabdfsdfssdfdsfด่ิอาห่สหด่สกห่ดสากหด่สหา


# #TODO 0

Convert the dataset to lowercase.

In [21]:
# Create a lowercase dataset for uncased BERT
def lower_case_sentences(examples):
    lower_cased_examples = examples
    
    # fill code here to lower case the "sentence" and "label_tokens"
    lower_cased_examples["sentence"]=lower_cased_examples["sentence"].lower()
    lower_cased_examples["label_tokens"] = [s.lower() for s in lower_cased_examples["label_tokens"]]
    
    
    return lower_cased_examples

In [22]:
orchidl = orchid.map(lower_case_sentences)

Map:   0%|          | 0/18500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4625 [00:00<?, ? examples/s]

In [23]:
orchidl

DatasetDict({
    train: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence'],
        num_rows: 4625
    })
})

In [24]:
orchidl["train"][1899]

{'id': '1899',
 'label_tokens': ['โดย',
  'พิจารณา',
  'จาก',
  'พจนานุกรม',
  'ภาษา',
  'คู่',
  ' ',
  '(',
  'bilingual transfer dictionary',
  ')'],
 'pos_tags': [25, 39, 38, 26, 26, 5, 37, 37, 26, 37],
 'sentence': 'โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)'}

Now let's examine the labels again.

In [25]:
example = orchidl["train"][1899]
print('sentence :', example["sentence"])
tokenized_input = tokenizer([example["sentence"]], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print('tokens :',tokens)
print('label tokens :', example["label_tokens"])
print('label pos :', example["pos_tags"])

sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)
tokens : ['<s>', '▁โดย', 'พิจารณาจาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '▁(', 'bi', 'ling', 'ual', '<_>', '▁', 'trans', 'fer', '<_>', '▁', 'di', 'ction', 'ary', ')', '</s>']
label tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'bilingual transfer dictionary', ')']
label pos : [25, 39, 38, 26, 26, 5, 37, 37, 26, 37]


In [26]:
example = orchidl["train"][0]
print('sentence :', example["sentence"])
tokenized_input = tokenizer([example["sentence"]], is_split_into_words=True)
print(tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print('tokens :',tokens)
print('label tokens :', example["label_tokens"])
print('label pos :', example["pos_tags"])

sentence : การประชุมทางวิชาการ ครั้งที่ 1
{'input_ids': [5, 10, 882, 8222, 8, 10, 1014, 8, 10, 59, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
tokens : ['<s>', '▁', 'การประชุม', 'ทางวิชาการ', '<_>', '▁', 'ครั้งที่', '<_>', '▁', '1', '</s>']
label tokens : ['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1']
label pos : [21, 39, 26, 26, 37, 4, 18]


In the example above, tokens refer to those tokenized using the pretrained tokenizer, while label tokens refer to tokens tokenized from our dataset.

**Do you see something?**

Yes, the tokens from the two tokenizers do not match.

- sentence : `การประชุมทางวิชาการ ครั้งที่ 1`

---

- tokens : `['<s>', '▁', 'การประชุม', 'ทางวิชาการ', '<_>', '▁', 'ครั้งที่', '<_>', '▁', '1', '</s>']`


---


- label tokens : `['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1']`
- label pos : `[21, 39, 26, 26, 37, 4, 18]`

You can see that in our label tokens, 'การ' has a POS tag of 21, and 'ประชุม' has a POS tag of 39. However, when we tokenize the sentence using WangchanBERTa, we get the token 'การประชุม'. What POS tag should we assign to this new token?

**What should we do ?**

Based on this example, we found that the tokens from the WangchanBERTa do not directly align with our label tokens. This means we cannot directly use the label POS tags. Therefore, we need to reassign POS tags to the tokens produced by WangchanBERTa tokenization. The method we will use is majority voting:
- If a token from the WangchanBERTa matches a label token exactly, we will directly assign the POS tag from the label POS.
- If the token generated overlaps or combines multiple label tokens, we assign the POS tag based on the number of characters in each token: If the token contains the most characters from any label token, we assign the POS tag from that label token.

**Example :**

    # "การประชุม" (9 chars) is formed from "การ" (3 chars) + "ประชุม" (6 chars).
    # "การ" has a POS tag of 21,
    # and "ประชุม" has a POS tag of 39.
    # Therefore, the POS tag for "การประชุม" is 39,
    # as "การประชุม" is derived more from the "ประชุม" part than from the "การ" part.

    # 'ทางวิชาการ' (10 chars) is formed from 'ทาง' (3 chars) + 'วิชาการ' (7 chars)
    # "ทาง" has a POS tag of 26,
    # and "วิชาการ" has a POS tag of 2.
    # Therefore, the POS tag for "ทางวิชาการ" is 2,
    # as "ทางวิชาการ" is derived more from the "ทาง" part than from the "วิชาการ" part.

# #TODO 1

`**Warning: Please be careful of <unk>, an unknown word token.**`

`**Warning: Please be careful of " ำ ", the 'am' vowel. WangchanBERTa's internal preprocessing replaces all " ำ " to 'ํ' and 'า'**`

Assigning the label -100 to the special tokens `[<s>]` and `[</s>]` and `[_]`  so they’re ignored by the PyTorch loss function (see [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html): ignore_index)

In [27]:
def majority_vote_pos(examples):

    ####################################################################################################################
    # TO DO: Since the tokens from the output of the pretrained tokenizer
    # do not match the tokens in the label tokens of the dataset,
    # the task is to create a function to determine the POS tags of the tokens generated by the pretrained tokenizer.
    # This should be done by referencing the POS tags in the label tokens. If a token partially overlaps with others,
    # the POS tag from the segment with the greater number of characters should be assigned.
    #
    # Example :
    # "การประชุม" (9 chars) is formed from "การ" (3 chars) + "ประชุม" (6 chars).
    # "การ" has a POS tag of 21,
    # and "ประชุม" has a POS tag of 39.
    # Therefore, the POS tag for "การประชุม" is 39,
    # as "การประชุม" is derived more from the "ประชุม" part than from the "การ" part.
    #
    # 'ทางวิชาการ' (10 chars) is formed from 'ทาง' (3 chars) + 'วิชาการ' (7 chars)
    # "ทาง" has a POS tag of 26,
    # and "วิชาการ" has a POS tag of 2.
    # Therefore, the POS tag for "ทางวิชาการ" is 2,
    # as "ทางวิชาการ" is derived more from the "ทาง" part than from the "วิชาการ" part.

    # tokenize word by pretrained tokenizer
    tokenized_inputs = tokenizer([examples["sentence"]], is_split_into_words=True)

    # FILL CODE HERE
    new_tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
    label_tokens = examples["label_tokens"]
    pos_tags = examples["pos_tags"]
    new_pos_result = []
    label_idx = 0
    char_idx = 0

    for w in new_tokens:
        if w in ["<s>", "</s>", "_"]:
            new_pos_result.append(-100)
            continue

        w = w.replace('ํา', "ำ")
        w = w.replace("<_>", " ")
        if w[0]=="▁":
            w = w[1:]

        if not w:
            new_pos_result.append(-100)
            continue

        while label_idx < len(label_tokens) and char_idx < len(label_tokens[label_idx]):
            if label_tokens[label_idx][char_idx] == w[0]:
                break
            char_idx += 1
            if char_idx >= len(label_tokens[label_idx]):
                label_idx += 1
                char_idx = 0

        buff = ""
        len_count = {}
        while buff != w and label_idx < len(label_tokens):
            buff += label_tokens[label_idx][char_idx]
            pos_tag = pos_tags[label_idx]

            if pos_tag not in len_count:
                len_count[pos_tag] = 0
            len_count[pos_tag] += 1

            char_idx += 1
            if char_idx >= len(label_tokens[label_idx]):
                label_idx += 1
                char_idx = 0

        max_ln=0
        max_ln_tag=0
        for k in len_count:
            if(len_count[k]>max_ln):
                max_ln=len_count[k]
                max_ln_tag=k
        new_pos_result.append(max_ln_tag)

    
    tokenized_inputs['tokens'] = new_tokens
    tokenized_inputs['labels'] = new_pos_result
    return tokenized_inputs
    ####################################################################################################################

In [28]:
tokenized_orchid = orchidl.map(majority_vote_pos)

Map:   0%|          | 0/18500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4625 [00:00<?, ? examples/s]

In [29]:
tokenized_orchid

DatasetDict({
    train: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence', 'input_ids', 'attention_mask', 'tokens', 'labels'],
        num_rows: 18500
    })
    test: Dataset({
        features: ['id', 'label_tokens', 'pos_tags', 'sentence', 'input_ids', 'attention_mask', 'tokens', 'labels'],
        num_rows: 4625
    })
})

In [30]:
tokenized_orchid['train'][0]

{'id': '0',
 'label_tokens': ['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1'],
 'pos_tags': [21, 39, 26, 26, 37, 4, 18],
 'sentence': 'การประชุมทางวิชาการ ครั้งที่ 1',
 'input_ids': [5, 10, 882, 8222, 8, 10, 1014, 8, 10, 59, 6],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'tokens': ['<s>',
  '▁',
  'การประชุม',
  'ทางวิชาการ',
  '<_>',
  '▁',
  'ครั้งที่',
  '<_>',
  '▁',
  '1',
  '</s>'],
 'labels': [-100, -100, 39, 26, 37, -100, 4, 18, -100, 18, -100]}

This is the result after we realigned the POS based on the majority vote.
- label_tokens : `['การ', 'ประชุม', 'ทาง', 'วิชาการ', ' ', 'ครั้ง', 'ที่ 1']`
- pos_tags : `[21, 39, 26, 26, 37, 4, 18]`
- tokens : `['<s>', '▁', 'การประชุม', 'ทางวิชาการ', '<_>', '▁', 'ครั้งที่', '<_>', '▁', '1', '</s>']`
- labels : `[-100, -100, 39, 26, 37, -100, 4, 18, -100, 18, -100]`

`['<s>', '▁', '</s>'] : -100`

**Check :**

> "การประชุม" (9 chars) is formed from "การ" (3 chars) + "ประชุม" (6 chars).


> "การ" has a POS tag of 21,

> and "ประชุม" has a POS tag of 39.

> Therefore, the POS tag for "การประชุม" is 39,

> as "การประชุม" is derived more from the "ประชุม" part than from the "การ" part.





In [31]:
# hard test case
example = tokenized_orchid["train"][1899]
for i in example :
    print(i, ":", example[i])

id : 1899
label_tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'bilingual transfer dictionary', ')']
pos_tags : [25, 39, 38, 26, 26, 5, 37, 37, 26, 37]
sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)
input_ids : [5, 489, 15617, 19737, 958, 493, 8, 1241, 4906, 11608, 12177, 8, 10, 11392, 9806, 8, 10, 2951, 15779, 8001, 29, 6]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens : ['<s>', '▁โดย', 'พิจารณาจาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '▁(', 'bi', 'ling', 'ual', '<_>', '▁', 'trans', 'fer', '<_>', '▁', 'di', 'ction', 'ary', ')', '</s>']
labels : [-100, 25, 39, 26, 26, 5, 37, 37, 26, 26, 26, 26, -100, 26, 26, 26, -100, 26, 26, 26, 37, -100]


Expected output


```
id : 1899
label_tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'bilingual transfer dictionary', ')']
pos_tags : [25, 39, 38, 26, 26, 5, 37, 37, 26, 37]
sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)
input_ids : [5, 489, 15617, 19737, 958, 493, 8, 1241, 4906, 11608, 12177, 8, 10, 11392, 9806, 8, 10, 2951, 15779, 8001, 29, 6]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens : ['<s>', '▁โดย', 'พิจารณาจาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '▁(', 'bi', 'ling', 'ual', '<_>', '▁', 'trans', 'fer', '<_>', '▁', 'di', 'ction', 'ary', ')', '</s>']
labels : [-100, 25, 39, 26, 26, 5, 37, 37, 26, 26, 26, 26, -100, 26, 26, 26, -100, 26, 26, 26, 37, -100]
```

# Train and Evaluate model

We will create a batch of examples using [DataCollatorWithPadding.](https://huggingface.co/docs/transformers/v4.48.0/en/main_classes/data_collator#transformers.DataCollatorWithPadding)  

Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.

DataCollatorWithPadding will help us pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length. This allows for efficient computation during each batch.

*   DataCollatorForTokenClassification : `padding (bool, str or PaddingStrategy, optional, defaults to True)`
*   `True` or `'longest'` (default): Pad to the longest sequence in the batch (or no padding if only a single sequence is provided).



In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

For evaluating your model’s performance. You can quickly load a evaluation method with the [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric). Seqeval actually produces several scores: precision, recall, F1, and accuracy.

In [33]:
import evaluate

seqeval = evaluate.load("seqeval")

Huggingface requires us to write a ``compute_metrics()`` function. This will be invoked when huggingface evalutes a model.

Note that we ignore to evaluate on -100 labels.

In [34]:
import numpy as np
import warnings


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

The total number of labels in our POS tag set.

In [35]:
id2label = {
    0: 'ADVI',
    1: 'ADVN',
    2: 'ADVP',
    3: 'ADVS',
    4: 'CFQC',
    5: 'CLTV',
    6: 'CMTR',
    7: 'CMTR@PUNC',
    8: 'CNIT',
    9: 'CVBL',
    10: 'DCNM',
    11: 'DDAC',
    12: 'DDAN',
    13: 'DDAQ',
    14: 'DDBQ',
    15: 'DIAC',
    16: 'DIAQ',
    17: 'DIBQ',
    18: 'DONM',
    19: 'EAFF',
    20: 'EITT',
    21: 'FIXN',
    22: 'FIXV',
    23: 'JCMP',
    24: 'JCRG',
    25: 'JSBR',
    26: 'NCMN',
    27: 'NCNM',
    28: 'NEG',
    29: 'NLBL',
    30: 'NONM',
    31: 'NPRP',
    32: 'NTTL',
    33: 'PDMN',
    34: 'PNTR',
    35: 'PPRS',
    36: 'PREL',
    37: 'PUNC',
    38: 'RPRE',
    39: 'VACT',
    40: 'VATT',
    41: 'VSTA',
    42: 'XVAE',
    43: 'XVAM',
    44: 'XVBB',
    45: 'XVBM',
    46: 'XVMM',
    # 47: 'O'
}
label2id = {}
for k, v in id2label.items() :
    label2id[v] = k

label2id

{'ADVI': 0,
 'ADVN': 1,
 'ADVP': 2,
 'ADVS': 3,
 'CFQC': 4,
 'CLTV': 5,
 'CMTR': 6,
 'CMTR@PUNC': 7,
 'CNIT': 8,
 'CVBL': 9,
 'DCNM': 10,
 'DDAC': 11,
 'DDAN': 12,
 'DDAQ': 13,
 'DDBQ': 14,
 'DIAC': 15,
 'DIAQ': 16,
 'DIBQ': 17,
 'DONM': 18,
 'EAFF': 19,
 'EITT': 20,
 'FIXN': 21,
 'FIXV': 22,
 'JCMP': 23,
 'JCRG': 24,
 'JSBR': 25,
 'NCMN': 26,
 'NCNM': 27,
 'NEG': 28,
 'NLBL': 29,
 'NONM': 30,
 'NPRP': 31,
 'NTTL': 32,
 'PDMN': 33,
 'PNTR': 34,
 'PPRS': 35,
 'PREL': 36,
 'PUNC': 37,
 'RPRE': 38,
 'VACT': 39,
 'VATT': 40,
 'VSTA': 41,
 'XVAE': 42,
 'XVAM': 43,
 'XVBB': 44,
 'XVBM': 45,
 'XVMM': 46}

In [36]:
labels = [i for i in id2label.values()]
labels

['ADVI',
 'ADVN',
 'ADVP',
 'ADVS',
 'CFQC',
 'CLTV',
 'CMTR',
 'CMTR@PUNC',
 'CNIT',
 'CVBL',
 'DCNM',
 'DDAC',
 'DDAN',
 'DDAQ',
 'DDBQ',
 'DIAC',
 'DIAQ',
 'DIBQ',
 'DONM',
 'EAFF',
 'EITT',
 'FIXN',
 'FIXV',
 'JCMP',
 'JCRG',
 'JSBR',
 'NCMN',
 'NCNM',
 'NEG',
 'NLBL',
 'NONM',
 'NPRP',
 'NTTL',
 'PDMN',
 'PNTR',
 'PPRS',
 'PREL',
 'PUNC',
 'RPRE',
 'VACT',
 'VATT',
 'VSTA',
 'XVAE',
 'XVAM',
 'XVBB',
 'XVBM',
 'XVMM']

## Load pretrained model

Select a pretrained model for fine-tuning to develop a POS Tagger model using the Orchid corpus dataset.



*   model : `wangchanberta-base-att-spm-uncased`
*   Don't forget to update the num_labels.

You’re ready to start training your model now! Load pretrained model with AutoModelForTokenClassification along with the number of expected labels, and the label mappings:




`In the first part, we require you to select the wangchanberta-base-att-spm-uncased.`

In [37]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
]

#@title Choose Pretrained Model
model_name = "wangchanberta-base-att-spm-uncased"

#create model
model = AutoModelForTokenClassification.from_pretrained(
    f"airesearch/{model_name}",
    revision='main',
    num_labels=47, id2label=id2label, label2id=label2id
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably T

### #TODO 2

* Configure your training hyperparameters using `**TrainingArguments**`. The only required parameter is is `output_dir`, which determines the directory where your model will be saved. To upload the model to the Hugging Face Hub, set push_to_hub=True (note: you must be logged into Hugging Face for this). During training, the Trainer will compute seqeval metrics at the end of each epoch and store the training checkpoint.
* Provide the `**Trainer**` with the training arguments, as well as the model, dataset, tokenizer, data collator, and compute_metrics function.
* Use `**train()**` to fine-tune the model.


Read [huggingface's tutorial](https://huggingface.co/docs/transformers/en/tasks/token_classification) for more details.

In [38]:
!git config --global user.email "theepob15112@gmail.com"
!git config --global user.name "Theepob"

In [52]:
training_args = TrainingArguments(
    #########################
    output_dir="pos-spm",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=True
    #########################
)

trainer = Trainer(
    #########################
    model=model,
    args=training_args,
    train_dataset=tokenized_orchid["train"],
    eval_dataset=tokenized_orchid["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    ########################
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/kaggle/working/pos-spm is already a clone of https://huggingface.co/Theepob/pos-spm. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.219000
1000,0.184400


TrainOutput(global_step=1158, training_loss=0.20093247145156992, metrics={'train_runtime': 275.4074, 'train_samples_per_second': 134.346, 'train_steps_per_second': 4.205, 'total_flos': 1207602506181672.0, 'train_loss': 0.20093247145156992, 'epoch': 2.0})

# Inference

With your model fine-tuned, you can now perform inference.

In [53]:
text = "การประชุมทางวิชาการ ครั้งที่ 1"

`In the first part, we require you to select the wangchanberta-base-att-spm-uncased.`

In [54]:
from transformers import AutoTokenizer

# Load pretrained tokenizer from Hugging Face
#@title Choose Pretrained Model
model_name = "airesearch/wangchanberta-base-att-spm-uncased"

tokenizer = Tokenizer(model_name).from_pretrained(model_name)
inputs = tokenizer(text, return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CamembertTokenizer'. 
The class this function is called from is 'WangchanbertaTokenizer'.


In [55]:
inputs

{'input_ids': tensor([[   5,   10,  882, 8222,    8,   10, 1014,    8,   10,   59,    6]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [56]:
from transformers import AutoModelForTokenClassification

## Load your fine-tuned model from Hugging Face
model = AutoModelForTokenClassification.from_pretrained("Theepob/pos-spm") ## your model path from Hugging Face
with torch.no_grad():
    logits = model(**inputs).logits

config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [57]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['PUNC',
 'PUNC',
 'VACT',
 'NCMN',
 'PUNC',
 'PUNC',
 'CFQC',
 'DONM',
 'DONM',
 'DONM',
 'PUNC']

In [58]:
id2label

{0: 'ADVI',
 1: 'ADVN',
 2: 'ADVP',
 3: 'ADVS',
 4: 'CFQC',
 5: 'CLTV',
 6: 'CMTR',
 7: 'CMTR@PUNC',
 8: 'CNIT',
 9: 'CVBL',
 10: 'DCNM',
 11: 'DDAC',
 12: 'DDAN',
 13: 'DDAQ',
 14: 'DDBQ',
 15: 'DIAC',
 16: 'DIAQ',
 17: 'DIBQ',
 18: 'DONM',
 19: 'EAFF',
 20: 'EITT',
 21: 'FIXN',
 22: 'FIXV',
 23: 'JCMP',
 24: 'JCRG',
 25: 'JSBR',
 26: 'NCMN',
 27: 'NCNM',
 28: 'NEG',
 29: 'NLBL',
 30: 'NONM',
 31: 'NPRP',
 32: 'NTTL',
 33: 'PDMN',
 34: 'PNTR',
 35: 'PPRS',
 36: 'PREL',
 37: 'PUNC',
 38: 'RPRE',
 39: 'VACT',
 40: 'VATT',
 41: 'VSTA',
 42: 'XVAE',
 43: 'XVAM',
 44: 'XVBB',
 45: 'XVBM',
 46: 'XVMM'}

In [59]:
# Inference
# ignore special tokens
text = 'จะว่าไปแล้วเชิงเทียนของผมก็สวยดีเหมือนกัน'
inputs = tokenizer(text, return_tensors="pt")
tokenized_input = tokenizer([text], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print('tokens :', tokens)
with torch.no_grad():
    logits = model(**inputs).logits
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
print('predict pos :', predicted_token_class)

tokens : ['<s>', '▁', 'จะว่าไป', 'แล้ว', 'เชิง', 'เทียน', 'ของ', 'ผมก็', 'สวยดี', 'เหมือนกัน', '</s>']
predict pos : ['NCMN', 'PUNC', 'ADVS', 'XVAE', 'NCMN', 'NCMN', 'RPRE', 'PPRS', 'VATT', 'ADVN', 'PUNC']


**Evaluate model :**

The output from the model is a softmax over classes. We choose the maximum class as the answer for evaluation. Again, we will ignore the -100 labels.

In [61]:
import pandas as pd
from IPython.display import display

def evaluation_report(y_true, y_pred, get_only_acc=False):
    # retrieve all tags in y_true
    tag_set = set()
    for sent in y_true:
        for tag in sent:
            tag_set.add(tag)
    for sent in y_pred:
        for tag in sent:
            tag_set.add(tag)
    tag_list = sorted(list(tag_set))

    # count correct points
    tag_info = dict()
    for tag in tag_list:
        tag_info[tag] = {'correct_tagged': 0, 'y_true': 0, 'y_pred': 0}

    all_correct = 0
    all_count = sum([len(sent) for sent in y_true])
    speacial_tag = 0
    for sent_true, sent_pred in zip(y_true, y_pred):
        for tag_true, tag_pred in zip(sent_true, sent_pred):
            # pass special token
            if tag_true == -100 :
                speacial_tag += 1
                pass
            if tag_true == tag_pred:
                tag_info[tag_true]['correct_tagged'] += 1
                all_correct += 1
            tag_info[tag_true]['y_true'] += 1
            tag_info[tag_pred]['y_pred'] += 1
    print('speacial_tag :',speacial_tag) # delete number of special token from all_count
    accuracy = (all_correct / (all_count-speacial_tag))

    # get only accuracy for testing
    if get_only_acc:
      return accuracy

    accuracy *= 100


    # summarize and make evaluation result
    eval_list = list()
    for tag in tag_list:
        eval_result = dict()
        eval_result['tag'] = tag
        eval_result['correct_count'] = tag_info[tag]['correct_tagged']
        precision = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_pred'])*100 if tag_info[tag]['y_pred'] else '-'
        recall = (tag_info[tag]['correct_tagged']/tag_info[tag]['y_true'])*100 if (tag_info[tag]['y_true'] > 0) else 0
        eval_result['precision'] = precision
        eval_result['recall'] = recall
        eval_result['f1_score'] = (2*precision*recall)/(precision+recall) if (type(precision) is float and recall > 0) else '-'

        eval_list.append(eval_result)

    eval_list.append({'tag': 'accuracy=%.2f' % accuracy, 'correct_count': '', 'precision': '', 'recall': '', 'f1_score': ''})

    df = pd.DataFrame.from_dict(eval_list)
    df = df[['tag', 'precision', 'recall', 'f1_score', 'correct_count']]

    display(df)


In [62]:
# prepare test set
test_data = tokenized_orchid["test"]

In [63]:
# labels for test set
y_test = []
for inputs in test_data:
  y_test.append(inputs['labels'])

In [64]:
y_pred = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for inputs in test_data:
    text = inputs['sentence']
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        pred = model(**inputs).logits
        predictions = torch.argmax(pred, dim=2)
        # Append padded predictions to y_pred
        y_pred.append(predictions.tolist()[0])

In [66]:
# check our prediction with label
# -100 is special tokens : [<s>, </s>, _]
print(y_pred[0])
print(y_test[0])

[26, 29, 39, 26, 26, 26, 37, 37, 26, 26, 26, 41, 26, 37, 26, 26, 39, 26, 37]
[-100, 29, 39, 26, 26, 26, 37, -100, 26, 26, 26, 41, 37, -100, 26, 26, 39, 26, -100]


In [67]:
evaluation_report(y_test, y_pred)

speacial_tag : 21042


,tag,precision,recall,f1_score,correct_count
0,-100,-,0.0,-,0
1,0,46.153846,37.5,41.37931,6
2,1,76.165803,72.772277,74.43038,735
3,2,25.0,6.034483,9.722222,7
4,3,63.265306,53.448276,57.943925,31
5,4,86.0,76.785714,81.132075,43
6,5,66.666667,49.710983,56.953642,86
7,6,74.836601,95.8159,84.036697,687
8,7,-,0.0,-,0
9,8,46.481481,63.705584,53.747323,251


# Other Pretrained model

In this section, we will experiment by fine-tuning other pretrained models, such as airesearch/wangchanberta-base-wiki-newmm, to see how about their performance.

Since each model uses a different word-tokenization method.
for example, **airesearch/wangchanberta-base-wiki-newmm uses newmm**,
while **airesearch/wangchanberta-base-att-spm-uncased uses SentencePiece**.
please try fine-tuning and compare the performance of these models.

### #TODO 3

In [68]:
model_names = [
    'airesearch/wangchanberta-base-att-spm-uncased',
    'airesearch/wangchanberta-base-wiki-newmm',
    'airesearch/wangchanberta-base-wiki-ssg',
    'airesearch/wangchanberta-base-wiki-sefr',
    'airesearch/wangchanberta-base-wiki-spm',
]

#@title Choose Pretrained Model
model_name = "airesearch/wangchanberta-base-wiki-newmm" #@param ["airesearch/wangchanberta-base-att-spm-uncased", "airesearch/wangchanberta-base-wiki-newmm", "airesearch/wangchanberta-base-wiki-syllable", "airesearch/wangchanberta-base-wiki-sefr", "airesearch/wangchanberta-base-wiki-spm"]

#create tokenizer
tokenizer = Tokenizer(model_name).from_pretrained(
                f'{model_name}',
                revision='main',
                model_max_length=416,)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


newmm.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/559 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'ThaiWordsNewmmTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'ThaiWordsNewmmTokenizer'.


In [69]:
example = orchidl["train"][1899]
print('sentence :', example["sentence"])
tokenized_input = tokenizer([example["sentence"]], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print('tokens :',tokens)
print('label tokens :', example["label_tokens"])

sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)
tokens : ['<s>', 'โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '<unk>', '<_>', 'transfer', '<_>', 'dictionary', ')', '</s>']
label tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'bilingual transfer dictionary', ')']


It's the same problem as above.

`**Warning: Can we use same function as above ?**`

`**Warning: Please beware of <unk>, an unknown word token.**`

`**Warning: Please be careful of " ำ ", the 'am' vowel. WangchanBERTa's internal preprocessing replaces all " ำ " to 'ํ' and 'า'**`

In [70]:
def majority_vote_pos(examples):

    ####################################################################################################################
    # TO DO: Since the tokens from the output of the pretrained tokenizer
    # do not match the tokens in the label tokens of the dataset,
    # the task is to create a function to determine the POS tags of the tokens generated by the pretrained tokenizer.
    # This should be done by referencing the POS tags in the label tokens. If a token partially overlaps with others,
    # the POS tag from the segment with the greater number of characters should be assigned.
    #
    # Example :
    # "การประชุม" (9 chars) is formed from "การ" (3 chars) + "ประชุม" (6 chars).
    # "การ" has a POS tag of 21,
    # and "ประชุม" has a POS tag of 39.
    # Therefore, the POS tag for "การประชุม" is 39,
    # as "การประชุม" is derived more from the "ประชุม" part than from the "การ" part.
    #
    # 'ทางวิชาการ' (10 chars) is formed from 'ทาง' (3 chars) + 'วิชาการ' (7 chars)
    # "ทาง" has a POS tag of 26,
    # and "วิชาการ" has a POS tag of 2.
    # Therefore, the POS tag for "ทางวิชาการ" is 2,
    # as "ทางวิชาการ" is derived more from the "ทาง" part than from the "วิชาการ" part.

    # FILL CODE HERE
    tokenized_inputs = tokenizer([examples["sentence"]], is_split_into_words=True)
    new_tokens = tokenizer.convert_ids_to_tokens(tokenized_inputs["input_ids"])
    label_tokens = examples["label_tokens"]
    pos_tags = examples["pos_tags"]
    
    new_pos_result = []
    label_idx = 0  # Current index in label_tokens
    char_idx = 0   # Current character index within the current label token

    for w in new_tokens:
        if w in ["<s>", "</s>", "_", "<unk>"]:
            new_pos_result.append(-100)
            continue
        
        w = w.replace('ํา', 'ำ')
        w = w.replace("<_>", " ")
        if w[0] == "▁":
            w = w[1:]
        
        if not w:
            new_pos_result.append(-100)
            continue
        
        len_count = {}
        buff = ""
        w_length = len(w)
        prev_buff_len = -1  

        while buff != w and label_idx < len(label_tokens):
            current_label = label_tokens[label_idx]
            remaining_in_label = current_label[char_idx:] if char_idx < len(current_label) else ''
            max_overlap = w_length - len(buff)
            overlap = min(len(remaining_in_label), max_overlap)
            
            if overlap == 0 and len(buff) == prev_buff_len:
                break 
            prev_buff_len = len(buff)
            
            buff += remaining_in_label[:overlap]
            for _ in range(overlap):
                pos_tag = pos_tags[label_idx]
                len_count[pos_tag] = len_count.get(pos_tag, 0) + 1
            char_idx += overlap
            
            if char_idx >= len(current_label):
                label_idx += 1
                char_idx = 0
        
        max_ln=0
        max_ln_tag=0
        for k in len_count:
            if(len_count[k]>max_ln):
                max_ln=len_count[k]
                max_ln_tag=k
        new_pos_result.append(max_ln_tag)
    
    tokenized_inputs['tokens'] = new_tokens
    tokenized_inputs['labels'] = new_pos_result
    return tokenized_inputs

    ####################################################################################################################

In [71]:
tokenized_orchid = orchidl.map(majority_vote_pos)

Parameter 'function'=<function majority_vote_pos at 0x791228ab2290> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/18500 [00:00<?, ? examples/s]

Map:   0%|          | 0/4625 [00:00<?, ? examples/s]

In [72]:
# hard test case
example = tokenized_orchid["train"][1899]
for i in example :
    print(i, ":", example[i])

id : 1899
label_tokens : ['โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', ' ', '(', 'bilingual transfer dictionary', ')']
pos_tags : [25, 39, 38, 26, 26, 5, 37, 37, 26, 37]
sentence : โดยพิจารณาจากพจนานุกรมภาษาคู่ (bilingual transfer dictionary)
input_ids : [0, 80, 3973, 45, 12252, 3496, 592, 5, 3, 5, 30055, 5, 63190, 178, 2]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens : ['<s>', 'โดย', 'พิจารณา', 'จาก', 'พจนานุกรม', 'ภาษา', 'คู่', '<_>', '<unk>', '<_>', 'transfer', '<_>', 'dictionary', ')', '</s>']
labels : [-100, 25, 39, 38, 26, 26, 5, 37, -100, 37, 26, 26, 26, 26, -100]


In [73]:
model_names = [
    'wangchanberta-base-att-spm-uncased',
    'wangchanberta-base-wiki-newmm',
    'wangchanberta-base-wiki-ssg',
    'wangchanberta-base-wiki-sefr',
    'wangchanberta-base-wiki-spm',
]

#@title Choose Pretrained Model
model_name = "wangchanberta-base-wiki-newmm" #@param ["wangchanberta-base-att-spm-uncased", "wangchanberta-base-wiki-newmm", "wangchanberta-base-wiki-syllable", "wangchanberta-base-wiki-sefr", "wangchanberta-base-wiki-spm"]

#create model
model = AutoModelForTokenClassification.from_pretrained(
    f"airesearch/{model_name}",
    revision='main',
    num_labels=47, id2label=id2label, label2id=label2id
)


pytorch_model.bin:   0%|          | 0.00/646M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [74]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

### #TODO 4

Fine-tuning other pretrained model with our orchid corpus.

In [75]:
training_args = TrainingArguments(
    #########################
    output_dir="wangchanberta-base-wiki-spm",
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    num_train_epochs=5,
    learning_rate=1e-4,
    push_to_hub=True
    #########################
)

trainer = Trainer(
    #########################
    model=model,
    args=training_args,
    train_dataset=tokenized_orchid["train"],
    eval_dataset=tokenized_orchid["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
    ########################
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Theepob/wangchanberta-base-wiki-spm into local empty directory.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.709000,0.510511,0.771984,0.765131,0.768542,0.834197
2,0.474300,0.497497,0.758728,0.775772,0.767155,0.837508
3,0.382500,0.505393,0.757609,0.772225,0.764847,0.837084
4,0.309900,0.546593,0.758952,0.781214,0.769922,0.840353
5,0.251400,0.577103,0.759344,0.778155,0.768635,0.839614


TrainOutput(global_step=2895, training_loss=0.3976514931581593, metrics={'train_runtime': 665.7523, 'train_samples_per_second': 138.941, 'train_steps_per_second': 4.348, 'total_flos': 2285504164317024.0, 'train_loss': 0.3976514931581593, 'epoch': 5.0})

In [76]:
######## EVALUATE YOUR MODEL ########
test_data = tokenized_orchid["test"]
model = AutoModelForTokenClassification.from_pretrained("Theepob/wangchanberta-base-wiki-spm")
y_test = []
for inputs in test_data:
  y_test.append(inputs['labels'])

y_pred = []
device = 'cuda' if torch.cuda.is_available() else 'cpu'
for inputs in test_data:
    text = inputs['sentence']
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        pred = model(**inputs).logits
        predictions = torch.argmax(pred, dim=2)
        # Append padded predictions to y_pred
        y_pred.append(predictions.tolist()[0])

print(y_pred[0])
print(y_test[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/643M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:463: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

[39, 29, 37, 21, 39, 26, 26, 37, 26, 26, 41, 37, 26, 39, 39, 26, 26]
[-100, 29, 37, 21, 39, 26, 26, 37, 26, 26, 41, 37, 26, 39, 39, 26, -100]


In [77]:
evaluation_report(y_test, y_pred)

speacial_tag : 11485


,tag,precision,recall,f1_score,correct_count
0,-100,-,0.0,-,0
1,0,43.478261,66.666667,52.631579,10
2,1,40.399002,61.363636,48.721805,648
3,2,14.0,12.5,13.207547,14
4,3,9.89011,31.578947,15.062762,18
5,4,72.881356,70.491803,71.666667,43
6,5,63.265306,58.125,60.586319,93
7,6,53.297683,67.191011,59.44334,299
8,7,90.909091,100.0,95.238095,10
9,8,54.293629,60.681115,57.309942,196


### #TODO 5

Compare the results between both models. Are they comparable? (Think about the ground truths of both models).

Propose a way to fairly evaluate the models.

<b>Write your answer here :</b>

### ANS
wangchanberta-base-att-spm-uncased มีข้อมูลที่ทั่วไปมากกว่า wangchanberta-base-wiki-spm ที่มาจากแค่wikipedia ข้อมูลจึงมีตวามเฉพาะตัวมาก wangchanberta-base-att-spm-uncased เลยชนะ wangchanberta-base-wiki-spm.

A note on preprocessing data.

``process_transformers`` in ``thaixtransformers.preprocess`` also provides a preprocess code that deals with many issues such as casing, text cleaning, and white space replacement with <_>. You can also use this to preprocess your text. Note that space replacement is done automatically without preprocessing in thaixtransformers.
